In [1]:
import pandas as pd
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
import time
from sklearn.model_selection import train_test_split
import numpy as np

pd.set_option('display.max_columns', None)

data=pd.read_csv(r'C:\Users\jisaa\Desktop\2008.csv\2008.csv')

In [2]:
templist=['Year','Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime',
          'ArrTime','CRSArrTime','UniqueCarrier','ActualElapsedTime',
          'CRSElapsedTime','AirTime','ArrDelay','DepDelay','Origin','Dest','Distance']
data=data[templist]
data=data.dropna(subset=templist)

In [3]:
data=data.rename(columns={'DepDelay':'y'})
data=data.loc[lambda x: x['y']>30,:]

templist=['ActualElapsedTime','CRSElapsedTime','AirTime',
          'ArrDelay','y','DepTime','CRSDepTime','ArrTime','CRSArrTime']
data.loc[:,templist] = data.loc[:,templist].astype(int)

templist=['DepTime','CRSDepTime','ArrTime','CRSArrTime']
for x in templist:
    data.loc[:,x]=pd.to_datetime(data.loc[:,x].astype(str),format='%H%M',errors='coerce').dt.hour

In [4]:
def cutoff(columnname,percentile):
    count=pd.DataFrame(data.groupby([columnname]).count()['Year'])
    count=count.sort_values(by=['Year'], ascending=False)
    count['cum_sum'] = count.Year.cumsum()
    count['cum_perc'] = 100*count.cum_sum/count.Year.sum()
    count=count.loc[count.cum_perc<percentile]
    return count.index.values

carrier75=cutoff('UniqueCarrier',75)
origin75=cutoff('Origin',75)
dest75=cutoff('Dest',75)
carrier=data.UniqueCarrier.values
origin=data.Origin.values
dest=data.Dest.values
newcarrier=[x if x in carrier75 else "Other" for x in carrier]
neworigin=[x if x in origin75 else "Other" for x in origin]
newdest=[x if x in dest75 else "Other" for x in dest]

data['UniqueCarrier']=newcarrier
data['Origin']=neworigin
data['Dest']=newdest

holiday=['2018-1-1','2018-1-21','2018-2-18','2018-5-26','2018-7-4','2018-9-1','2018-11-27','2018-11-28','2018-12-24','2018-12-25','2018-12-31']
data['date']=data['Year'].astype(str)+"-"+data['Month'].astype(str)+"-"+data['DayofMonth'].astype(str)
data['holiday']=data['date'].isin(holiday)

In [5]:
#
data=data.sample(frac=0.1)
data=data.reset_index()
#

templist=['Month','DayofMonth','DayOfWeek','DepTime','CRSDepTime','ArrTime',
          'CRSArrTime','holiday','UniqueCarrier','Origin','Dest']
dummyx=data[templist]
dummyx=dummyx.astype('category')
dummyx= pd.get_dummies(dummyx)

templist=['ArrDelay','ActualElapsedTime', 'CRSElapsedTime', 'AirTime','ArrDelay']
continuousx=data[templist]

X=pd.concat([dummyx, continuousx], axis=1, sort=True)
y=data['y']

In [6]:
X.train,X.test,y.train,y.test=train_test_split(X, y, test_size = 0.25)

rf=ensemble.RandomForestRegressor(n_estimators=100)
rf.fit(X.train, y.train)

C:\Users\jisaa\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [7]:
y.pred= rf.predict(X.test)
errors = abs(y.pred - y.test)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 2.12 degrees.
